In [2]:
## Getting the data and loading packages
library(tidyverse)
library(stringr)
library(ggmap)
library(forcats)
library(rvest)

Loading required package: xml2
Warning message:
“package ‘xml2’ was built under R version 3.3.2”
Attaching package: ‘rvest’

The following object is masked from ‘package:purrr’:

    pluck

The following object is masked from ‘package:readr’:

    guess_encoding



In [9]:
## Scraping data from Wikipedia
html.world_ports <- read_html("https://en.wikipedia.org/wiki/List_of_busiest_container_ports")
df.world_ports <- html_table(html_nodes(html.world_ports, "table")[[1]], fill=TRUE)

In [10]:
## inspecting
glimpse(df.world_ports)

Observations: 50
Variables: 18
$ `#`          <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,...
$ Port         <chr> "Shanghai", "Singapore", "Shenzhen", "Ningbo-Zhoushan"...
$ Jurisdiction <chr> "China", "Singapore", "China", "China", "South Korea",...
$ Region       <chr> "East Asia", "Southeast Asia", "East Asia", "East Asia...
$ Location     <chr> "Yangtze Delta", "Malacca Strait", "Pearl River Delta"...
$ `2016`       <chr> "37,133[12]", "30,904[13]", "23,979[14]", "21,560[15]"...
$ `2015[1]`    <chr> "36,537[12]", "30,922[13]", "24,204[14]", "20,620[15]"...
$ `2014[1]`    <chr> "35,268", "33,869", "23,798", "19,450", "18,423", "22,...
$ `2013[2]`    <chr> "33,617", "32,240", "23,280", "17,351", "17,690", "22,...
$ `2012[3]`    <chr> "32,529", "31,649", "22,940", "16,670", "17,046", "23,...
$ `2011[4]`    <chr> "31,700", "29,937", "22,570", "14,686", "16,185", "24,...
$ `2010[5]`    <chr> "29,069", "28,431", "22,510", "13,144", "14,157", "23,...
$ `2009[6]`    <chr> 

In [12]:
## Renaming the variables
 # transform names to lowercase
colnames(df.world_ports) <- colnames(df.world_ports) %>% tolower()

# inspect
colnames(df.world_ports)

[1] "#"            "port"         "jurisdiction" "region"       "location"    
 [6] "2016"         "2015[1]"      "2014[1]"      "2013[2]"      "2012[3]"     
[11] "2011[4]"      "2010[5]"      "2009[6]"      "2008[7]"      "2007[8]"     
[16] "2006[9]"      "2005[10]"     "2004[11]"

In [18]:
# Get geospatial information (longitude/latitude)
geocode.world_ports <- geocode(df.world_ports$port)

Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Shanghai&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Singapore&sensor=false
Warning message:
“geocode failed with status OVER_QUERY_LIMIT, location = "Singapore"”Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Shenzhen&sensor=false
Warning message:
“geocode failed with status OVER_QUERY_LIMIT, location = "Shenzhen"”Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Ningbo-Zhoushan&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Busan&sensor=false
Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Hong%20Kong&sensor=false
Warning message:
“geocode failed with status OVER_QUERY_LIMIT, location = "Hong Kong"”Information from URL : http://maps.googleapis.com/maps/api/geocode/json?address=Guangzhou&sensor=false
Warning message:
“ge

In [21]:
head(geocode.world_ports)

lon,lat
121.4737,31.23039
NA,NA
NA,NA
121.9878,29.90195
129.0756,35.17955
NA,NA


In [22]:
#--------------------------------------------------------
# COMBINE:
# - bind the new lat/long data to df.world_ports data frame
#--------------------------------------------------------
df.world_ports <- cbind(df.world_ports, geocode.world_ports)
head(df.world_ports)

#,port,jurisdiction,region,location,2016,2015[1],2014[1],2013[2],2012[3],2011[4],2010[5],2009[6],2008[7],2007[8],2006[9],2005[10],2004[11],lon,lat
1,Shanghai,China,East Asia,Yangtze Delta,"37,133[12]","36,537[12]","35,268","33,617","32,529","31,700","29,069","25,002","27,980","26,150","21,710","18,084","14,557",121.4737,31.23039
2,Singapore,Singapore,Southeast Asia,Malacca Strait,"30,904[13]","30,922[13]","33,869","32,240","31,649","29,937","28,431","25,866","29,918","27,932","24,792","23,192","21,329",NA,NA
3,Shenzhen,China,East Asia,Pearl River Delta,"23,979[14]","24,204[14]","23,798","23,280","22,940","22,570","22,510","18,250","21,414","21,099","18,469","16,197","13,615",NA,NA
4,Ningbo-Zhoushan,China,East Asia,Yangtze Delta,"21,560[15]","20,620[15]","19,450","17,351","16,670","14,686","13,144","10,502","11,226","9,349","7,068","5,208","4,006",121.9878,29.90195
5,Busan,South Korea,East Asia,Korean Strait,"19,850[16]","19,469[16]","18,423","17,690","17,046","16,185","14,157","11,954","13,425","13,270","12,039","11,843","11,430",129.0756,35.17955
6,Hong Kong,Hong Kong SAR,East Asia,Pearl River Delta,"19,813[17]","20,073[17]","22,374","22,352","23,117","24,384","23,532","20,983","24,248","23,881","23,539","22,427","21,984",NA,NA


In [23]:
## Manually code missing latitude/longitude data
#=========================================================================================
# RECODE lon and lat
# - There are 4 lon/lat values that weren't found with geocode()
# - We'll just hand code them
# - The values can be obtained at http://www.latlong.net/convert-address-to-lat-long.html
#
#
# Tanjung Pelepas, Johor Bahru: lon = 103.551035, lat = 1.362374
# Yingkou:..................... lon = 122.108231, lat = 40.266062
# Valencia, Spain:............. lon = -0.3762881, lat = 39.46991
# Malta Freeport:.............. lon = 14.537637 , lat = 35.816287
#
#=========================================================================================

df.world_ports <- df.world_ports %>%
                        mutate( lon = case_when(.$port == "Tanjung Pelepas" ~ 103.551035
                                                ,.$port == "Yingkou"        ~ 122.108231
                                                ,.$port == "Valencia"       ~ -0.3762881
                                                ,.$port == "Malta Freeport" ~ 14.537637
                                                ,.$port == "Port Said" ~ 32.301866
                                                ,.$port == "Nanjing" ~ 118.796877
                                                ,TRUE ~ .$lon
                                                )
                                ,lat = case_when(.$port == "Tanjung Pelepas" ~ 1.362374
                                                ,.$port == "Yingkou"        ~ 40.266062
                                                ,.$port == "Valencia"       ~ 39.46991
                                                ,.$port == "Malta Freeport" ~ 35.816287
                                                ,.$port == "Port Said" ~ 31.265289
                                                ,.$port == "Nanjing" ~ 32.060255
                                                ,TRUE ~ .$lat
                                                )
                                )

# CHECK
df.world_ports %>% filter(port == "Tanjung Pelepas") %>% select(lat,lon)
df.world_ports %>% filter(port == "Yingkou") %>% select(lat,lon)
df.world_ports %>% filter(port == "Valencia") %>% select(lat,lon)
df.world_ports %>% filter(port == "Malta Freeport") %>% select(lat,lon)


Warning message:
“package ‘bindrcpp’ was built under R version 3.3.2”

lat,lon
1.362374,103.551


lat,lon
40.26606,122.1082


lat,lon
39.46991,-0.3762881


lat,lon


In [24]:
df.world_ports

#,port,jurisdiction,region,location,2016,2015[1],2014[1],2013[2],2012[3],2011[4],2010[5],2009[6],2008[7],2007[8],2006[9],2005[10],2004[11],lon,lat
1,Shanghai,China,East Asia,Yangtze Delta,"37,133[12]","36,537[12]","35,268","33,617","32,529","31,700","29,069","25,002","27,980","26,150","21,710","18,084","14,557",121.4737021,31.230390
2,Singapore,Singapore,Southeast Asia,Malacca Strait,"30,904[13]","30,922[13]","33,869","32,240","31,649","29,937","28,431","25,866","29,918","27,932","24,792","23,192","21,329",NA,NA
3,Shenzhen,China,East Asia,Pearl River Delta,"23,979[14]","24,204[14]","23,798","23,280","22,940","22,570","22,510","18,250","21,414","21,099","18,469","16,197","13,615",NA,NA
4,Ningbo-Zhoushan,China,East Asia,Yangtze Delta,"21,560[15]","20,620[15]","19,450","17,351","16,670","14,686","13,144","10,502","11,226","9,349","7,068","5,208","4,006",121.9878356,29.901952
5,Busan,South Korea,East Asia,Korean Strait,"19,850[16]","19,469[16]","18,423","17,690","17,046","16,185","14,157","11,954","13,425","13,270","12,039","11,843","11,430",129.0756416,35.179554
6,Hong Kong,Hong Kong SAR,East Asia,Pearl River Delta,"19,813[17]","20,073[17]","22,374","22,352","23,117","24,384","23,532","20,983","24,248","23,881","23,539","22,427","21,984",NA,NA
7,Guangzhou,China,East Asia,Pearl River Delta,"18,858[18]","17,625","16,160","15,309","14,744","14,400","12,550","11,190","11,001","9,200","6,600","4,685","3,308",NA,NA
8,Qingdao,China,East Asia,Yellow Sea,"18,010[19]","17,510","16,624","15,520","14,503","13,020","12,012","10,260","10,320","9,462","7,702","6,307","5,140",120.3826090,36.067108
9,Dubai,United Arab Emirates,Western Asia,Arab Peninsula,"14,772[20]","15,592","14,750","13,641","13,270","13,000","11,600","11,124","11,827","10,653","8,923","7,619","6,429",NA,NA
10,Tianjin,China,East Asia,Yellow Sea,"14,490[21]","14,090","14,050","13,010","12,300","11,500","10,080","8,700","8,500","7,103","5,950","4,801","3,814",117.3616476,39.343357
